# Intro to ThermoFun and Reaktoro: Modeling the formation of stratiform, sediment-hosted Cu(-Co) deposits

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/cardinalgeo/hydrothermal-geochemistry-modeling/blob/main/Beer_Seminar_Geochem_Model.ipynb)
  

## Before we start...setting up coding environment

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install_miniforge()

Add libraries that aren't already in the Colab coding environment. 

In [ ]:
!conda install reaktoro
!conda install thermofun
!conda install thermohubclient

## What to expect
- Introduction to modeling framework (Python, ThermoFun & Reaktoro, and Jupyter Notebooks)
- How thermodynamic data are stored and accessed
- Geochemical modeling example
- **Not** a presentation on thermodynamics theory
    

## A brief introduction
### Why **Python**? 
- readable
- open-source $\rightarrow$ free

### Two main librarires
- **ThermoFun** (Dan Miron @ Paul Scherrer Institute, Switzerland)
    - calculate thermodynamic properties ($\Delta H$, $S$, $\Delta G$, $V$, $C_p$, etc.) of a substance or reaction @ T,P
- **Reaktoro** (Allan Leal @ ETH Zurich, Switzerland)
    - model (geo)chemical processes involving thermodynamics and kinetics
    - Gibbs Energy Minimization (**GEM**) approach (e.g., HCh, GEM-Selektor)
        - Law of Mass Action (**LMA**) (e.g., PHREEQC) $\rightarrow$ **GEM**

| Substances                                                           | Reactions                               | Fluid EoS                  | $H_2O$ EoS                                        |   |
|:---------------------------------------------------------------------|:----------------------------------------|:---------------------------|:-----------------------------------------------|:--|
| Birch-Muraghan, Cp and V integration                                 | Marshall-Franck density model           | Churakov-Gottschalk (CG)   | Haar-Gallagher-Kell (HGK)                      |   |
| Berman dVdP                                                          | **Modified Ryzhenko-Bryzgalin (MRB) model** | Peng-Robinson              | IAPWS Formulations-1995                        |   |
| **HKF (Helgeson-Kirkham-Flowers)**                                       | Dolejs and Manning (2010) density model | Peng-Robinson-Stryjek-Vera | PVT Zhang and Duan (2005)                      |   |
| Akinfiev-Diamond nonelectrolyte model                                |                                         | Compensated Redlich-Kwong  | Ideal Gas Wolley                               |   |
| Holland and Powell (1998) modified density model for aqueous species |                                         | Soave-Redlich-Kwong        | Dielectric constant: Johnson and Norton (1992) |   |
| Anderson et al. (1991) density model for aqueous species             |                                         | Sterner-Pitzer             | Dielectric constant: Fernandez et al. (1997)   |   |
|                                                                      |                                         |                            | Dielectric constant: Sverjensky et al. (2014)                       |   |

### Jupyter Notebook
- document containing code interpersed with text, equations, images, etc. 
- code and text cells

## Why consider an alternative to HCh? ### 
- HCh is...
    - no longer being actively maintained
    - Windows OS-only

### An alternative that is...
- open source
- actively maintained and improved
- accessible and flexible
- reproducible
- can be coupled with other libraries in the Python ecosystem
- **the result**: 
    - *accessible narrative or recipe that anyone can run*

# Demo: code and text cells

## Python demo

In [ ]:
a = 5 # J/mol
b = 2
c = a + b
# print(a + b)
print(c)

In [ ]:
# this is the same as before
print(c)

In [ ]:
def myFunction(x): 
    y = x + 2
    return y
print(myFunction(2))

## Markdown demo

Reaction: $Ni + \frac{1}{2}O_2 \rightarrow NiO$

## On to the modeling! 

In this next section, I'll walk through a simple model for the formation of **stratiform, sediment-hosted Cu(-Co) deposits**. Along the way, we'll see how to: 
- import Reaktoro and ThermoFun
- create a thermodynamic database from a data file
- create a chemical system
- add substances to that system
- equilibrate! 

## Import libraries

In [ ]:
import reaktoro as rkt # Gibbs free energy minimization (GEM)
import thermofun as fun # thermo. properties at T,P
import thermohubclient as client # get thermo. data

import numpy as np # facilitate math 
import matplotlib.pyplot as plt # plotting

## Set up a thermodynamic database

In [ ]:
# download data file 
dbc = client.DatabaseClient()
dbc.saveDatabase("mines16")

# create database object from data file
database = fun.Database('./mines16-thermofun.json')

Because the pyrrhotite entry in the mines16 data file is missing a critical piece of information—an issue I've submitted a fix for but which the developer has yet to resolve—I will instead use a data file stored in my google drive. To access it, I will define and then use the function `folder_download`, which will access the data file's parent folder and download its contents to Colab. The function is copied from [here](https://gist.github.com/korakot/51a917e1f53891d53be223439b0f75c1). 

In [ ]:
import requests # This library is used to make requests to internet

url = 'https://drive.google.com/file/d/13IDVpjXU4UM_X-f-KT6gf47UQ-gxDQNC/view?usp=sharing'
url = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]

# We are creating a requests variable with the above url
r = requests.get(url, allow_redirects=True)
# We are writing the content of above request to mines16-thermofun.json
open('./mines16-thermofun.json', 'wb').write(r.content);

# create database object from data file
database = fun.Database('./mines16-thermofun.json')


In [ ]:
import json 
with open('./mines16-thermofun.json') as json_file:
    mines16 = json.load(json_file)

Let's first take a look at how the `mines16-thermofun.json` data file is formatted. 
- generic `JSON` format
- collection of name-value pairs 
- name is a string; value can be a
    - string: `{"name": "pyrite"}`
    - number: `{"charge": -2}`
    - list: `{"name": ["Sc", "Co", "U", "Au"]}`
    - combos of the above: `{"Cp_coeff": {"value": [5, 10], "units": ["J/mol*K", "J/mol"}]`
- `JSON` is **flexible** and therefore **widespread**

In [ ]:
import uuid
from IPython.display import display_javascript, display_html, display
import json

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json
        self.uuid = str(uuid.uuid4())
        
    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 600px; width:100%;"></div>'.format(self.uuid),
            raw=True
        )
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
          renderjson.set_show_to_level(1)
          document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)

In [ ]:
RenderJSON(mines16)

## Check the contents of the database (optional)

In [ ]:
substances = database.mapSubstances()
print(substances.keys())

## 3. Define and construct the chemical system

In [ ]:
# Define the chemical system
editor = rkt.ChemicalEditor(database)

editor.addAqueousPhaseWithElements([
    'H', 'O', 'C', 'Na', 'Ca', 'Cl', 'S', 'Fe', 'Cu'])
editor.addMineralPhase([
    'Calcite', 'Anhydrite', 'Pyrite', 'Covellite', 'Chalcopyrite', 'Graphite'])

# Construct the chemical system
system = rkt.ChemicalSystem(editor)

Below we'll define a few useful functions. `removeFluid` sets the amounts of the the species in the fluid phase to zero moles – in effect, "removing" the fluid. `removeSolid` does the same, except for solid phases. 

In [ ]:
def removeFluid(state):
    system = state.system()
    idxs = system.indicesFluidSpecies()
    for i in idxs:
        state.setSpeciesAmount(i, 0.0)

def removeSolid(state): 
    system = state.system()
    idxs = system.indicesSolidSpecies()
    for i in idxs:
        state.setSpeciesAmount(i, 0.0)

For convenience, we'll also define two variables, `T` and `P`, for temperature and pressure. We'll use these variables throughout the next few cells whenever we need to set the PT conditions. 

In [ ]:
# set temperature and pressure
T = 150 # [°C]
P = rkt.waterSaturatedPressureWagnerPruss(T+273.15).val/100000 # [bar] saturated vapor pressure of water

### Create ore fluid
The fluids that form SSH Cu deposits are generally assumed to be oxidized chlorine-bearing brines derived either from the dissolution of evaporites or as a residual product of evaporation (i.e., bittern brines). We can simulate such an ore fluid by saturating a NaCl-bearing fluid with respect to anhydrite (e.g., $CaSO_{4(s)} \leftrightarrow Ca^{+2}_{(aq)} + SO4^{-2}_{(aq)}$) and then having it "scavenge" Cu (i.e., add $CuCl_{2(aq)}$). 

In [ ]:
state_fluid = rkt.ChemicalState(system)

# set temperature and pressure
state_fluid.setTemperature(T, "celsius")
state_fluid.setPressure(P, "bar")

# create fluid
state_fluid.setSpeciesMass("H2O@", 10, "kg")
state_fluid.setSpeciesAmount("NaCl@", 0.5, "mol")
state_fluid.setSpeciesMass("Anhydrite", 100, "kg")
state_fluid.setSpeciesMass("CuCl2@", 1.5e-2, "kg")

# equilibrate solids and fluid
rkt.equilibrate(state_fluid)

# "migrate" fluid from source
removeSolid(state_fluid)

### Create "sink"
SSH Cu deposits often co-occur with strong lithological redox gradients. In this model, we'll use a graphite and pyrite-bearing limestone to reduce the fluid and induce the precipitation of Cu ore. 

To represent this process, we'll use a **flush model**. An aliquot of the ore fluid will equilibrate with an aliquot of the limestone. After equilibration, the "spent" fluid will be removed from the system and the process will be repeated.

In [ ]:
# initialize solid 
state_sink = rkt.ChemicalState(system)

# set temperature and pressure
state_sink.setTemperature(T, "celsius")
state_sink.setPressure(P, "bar")

# create "rock"
state_sink.setSpeciesMass("Calcite", 97.5, "kg")
state_sink.setSpeciesMass("Pyrite", 2, "kg")
state_sink.setSpeciesMass("Graphite", 0.5, "kg")

# initialize "amount" arrays 
pyrite = np.zeros(80)
graphite = np.zeros(80)
covellite = np.zeros(80)
chalcopyrite = np.zeros(80)

for i in range(80):

    problem = rkt.EquilibriumProblem(system)
    problem.add(state_sink) # add composition for our rock
    problem.add(state_fluid) # add composition for our fluid

    state_sink = rkt.equilibrate(problem)

    removeFluid(state_sink)
    
    # save the amounts of the minerals of interest
    pyrite[i] = state_sink.speciesAmount("Pyrite")
    graphite[i] = state_sink.speciesAmount("Graphite")
    covellite[i] = state_sink.speciesAmount("Covellite")
    chalcopyrite[i] = state_sink.speciesAmount("Chalcopyrite")

In [ ]:
fig, ax  = plt.subplots(figsize=[12,8]) 

ax.plot(np.arange(i+1), covellite)
ax.plot(np.arange(i+1), chalcopyrite)
ax.plot(np.arange(i+1), pyrite)
ax.plot(np.arange(i+1), graphite)

ax.legend(["covellite", "chalcopyrite", "pyrite", "graphite"])
ax.set_xlabel('flush iterations')
ax.set_ylabel('amount of mineral (mol)')